In [106]:
import pandas as pd
import numpy as np
from pathlib import Path
import math
import os
from datetime import datetime

In [113]:
# Set folder path
data_folder = Path("task3_dataset/")
vibration_folder = Path(data_folder/"vibrationdata/")

# Set file paths
prod_log_path = data_folder / "production_log.csv"
prod_qual_path = data_folder / "product_quality_log.csv"

# Read production log and quality log files
df_prod_log = pd.read_csv(prod_log_path, index_col = 0)
df_qual_log = pd.read_csv(prod_qual_path, index_col = 0)

# Convert the timestamp format for uniformity
df_prod_log['timestamp'] = pd.to_datetime(df_prod_log['timestamp'], format = '%Y-%m-%d_%H-%M-%S')


In [114]:
display(df_prod_log)
display(df_qual_log)

,timestamp,product_id
0,2021-05-17 08:12:48,P3.2.500
1,2021-05-17 08:12:51,P3.2.501
2,2021-05-17 08:12:54,P3.2.502
3,2021-05-17 08:12:57,P3.2.503
4,2021-05-17 08:13:00,P3.2.504
...,...,...
1651,2021-05-17 09:35:21,P3.2.2151
1652,2021-05-17 09:35:24,P3.2.2152
1653,2021-05-17 09:35:27,P3.2.2153
1654,2021-05-17 09:35:30,P3.2.2154


,machine_id,product_id,quality
0,Printer F0815,P3.2.500,OK
1,Printer F0815,P3.2.501,OK
2,Printer F0815,P3.2.502,OK
3,Printer F0815,P3.2.503,OK
4,Printer F0815,P3.2.504,OK
...,...,...,...
1651,Printer F0815,P3.2.2151,nOK
1652,Printer F0815,P3.2.2152,nOK
1653,Printer F0815,P3.2.2153,nOK
1654,Printer F0815,P3.2.2154,nOK


In [117]:
# Merging product quality log file data and production log file data. Key = 'product_id'

df_prod_qual = df_prod_log.merge(df_qual_log, on = 'product_id', how = 'inner')
df_prod_qual = df_prod_qual.set_index('timestamp')

In [118]:
display(df_prod_qual)

,product_id,machine_id,quality
timestamp,,,
2021-05-17 08:12:48,P3.2.500,Printer F0815,OK
2021-05-17 08:12:51,P3.2.501,Printer F0815,OK
2021-05-17 08:12:54,P3.2.502,Printer F0815,OK
2021-05-17 08:12:57,P3.2.503,Printer F0815,OK
2021-05-17 08:13:00,P3.2.504,Printer F0815,OK
...,...,...,...
2021-05-17 09:35:21,P3.2.2151,Printer F0815,nOK
2021-05-17 09:35:24,P3.2.2152,Printer F0815,nOK
2021-05-17 09:35:27,P3.2.2153,Printer F0815,nOK


In [104]:
# print(df_prod_qual[df_prod_qual['quality'] == 'OK'].count())
# df_prod_qual[df_prod_qual['quality'] == 'nOK'].count()

In [123]:
#Test code - IGNORE
file = "2021-05-17_08-12-48"
vib_df1 = pd.read_csv(vibration_folder/file, sep = "\t", header = None)
vib_df1.columns = ['Sensor1','Sensor2']

## Functions for Feature Extraction/Creation

In [124]:
# Features extracted for each sensor:
# 1.Max 2.Min 3.Mean 4.Standard Deviation 5.Root Mean Square

def extractFeature(df):
    rms_sensor1 = math.sqrt((df['Sensor1'] ** 2).mean())
    rms_sensor2 = math.sqrt((df['Sensor2'] ** 2).mean())
    feature_array = np.array([ df['Sensor1'].max(), df['Sensor2'].max(),df['Sensor1'].min(),df['Sensor2'].min(),df['Sensor1'].mean(), df['Sensor2'].mean(),df['Sensor1'].std(),df['Sensor2'].std(), rms_sensor1, rms_sensor2])
    return feature_array

In [126]:
# Test code- IGNORE
feature_array = extractFeature(vib_df1)
print(feature_array)

[ 0.271       0.254      -0.75       -0.488      -0.11753828 -0.11665913
  0.09280735  0.0794944   0.14975991  0.141168  ]


### Reading vibration data file and creating feature matrix

In [102]:
feature_df = pd.DataFrame([])

for file in os.listdir(vibration_folder):
    vib_data = pd.read_csv(vibration_folder/file, sep = "\t", header = None)
    vib_data.columns = ['Sensor1','Sensor2']

    #Function call to extract features
    feature_array = extractFeature(vib_data)
    vib_df = pd.DataFrame([feature_array])
    
    #Converting string filename to datetime object
    file_datetime = datetime.strptime(str(file), '%Y-%m-%d_%H-%M-%S')
    vib_df.index = [file_datetime]
    
    feature_df = feature_df.append(vib_df)

In [119]:
feature_df.columns = ['S1_max', 'S2_max','S1_min','S2_min','S1_mean','S2_mean','S1_std','S2_std','S1_rms','S2_rms']

In [127]:
df_merged = pd.merge(df_prod_qual, feature_df, left_index = True, right_index = True)

In [128]:
display(df_merged)

,product_id,machine_id,quality,S1_max,S2_max,S1_min,S2_min,S1_mean,S2_mean,S1_std,S2_std,S1_rms,S2_rms
2021-05-17 08:12:48,P3.2.500,Printer F0815,OK,0.271,0.254,-0.750,-0.488,-0.117538,-0.116659,0.092807,0.079494,0.149760,0.141168
2021-05-17 08:12:51,P3.2.501,Printer F0815,OK,0.269,0.229,-0.620,-0.505,-0.116943,-0.115859,0.093839,0.079616,0.149936,0.140576
2021-05-17 08:12:54,P3.2.502,Printer F0815,OK,0.278,0.276,-0.540,-0.496,-0.117447,-0.117435,0.094562,0.081349,0.150782,0.142858
2021-05-17 08:12:57,P3.2.503,Printer F0815,OK,0.273,0.234,-0.742,-0.569,-0.116196,-0.116182,0.094652,0.080029,0.149867,0.141077
2021-05-17 08:13:00,P3.2.504,Printer F0815,OK,0.305,0.239,-0.632,-0.483,-0.116490,-0.116388,0.094036,0.079940,0.149707,0.141196
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-17 09:35:21,P3.2.2151,Printer F0815,nOK,0.466,0.388,-0.732,-0.620,-0.117700,-0.117854,0.145685,0.117771,0.187287,0.166610
2021-05-17 09:35:24,P3.2.2152,Printer F0815,nOK,0.518,0.488,-0.781,-0.598,-0.116901,-0.117200,0.146956,0.117993,0.187779,0.166306
2021-05-17 09:35:27,P3.2.2153,Printer F0815,nOK,0.547,0.496,-0.776,-0.830,-0.116428,-0.116559,0.164493,0.142499,0.201525,0.184095
2021-05-17 09:35:30,P3.2.2154,Printer F0815,nOK,0.728,0.552,-0.869,-0.791,-0.118304,-0.118188,0.170994,0.150242,0.207927,0.191155
